In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285398 sha256=51710715ab7f10c568b5b2398e0285cd0ca1fe6d0a2ddd0e6c0b516145054509
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


In [ ]:
#path = '/content/drive/MyDrive/Big Data/'

import os
path = "drive/MyDrive/Ciclos/2023-1/Big Data/ProyectoBD/Big Data/"
print(os.listdir(path))

['COVID19_Tweets_Dataset_Pq2', 'COVID19_Tweets_Dataset_Parquet', 'COVID19_Tweets_Dataset_Pq', 'COVID19_Tweets_Dataset_Pq3']


In [ ]:
import pandas as pd
import os

parte = "00"
dia = "01"

parte_max = 24
dia_max = 2

tables = ["Summary_Details", "Summary_Hashtag", "Summary_Mentions", "Summary_NER", "Summary_NER_ES", "Summary_Sentiment", "Summary_Sentiment_ES"]
# tables = ["Summary_NER_ES"]

for table in tables:
  for d in range(1, dia_max):
    for p in range(0, parte_max):
      try:
        if d < 10:
          dia = "0"+str(d)
        else:
          dia = str(d)
        if p < 10:
          parte = "0"+str(p)
        else:
          parte = str(p)

        if table == "Summary_NER_ES":
          file = "2022_01_" + dia + "_" + parte + "_Summary_ES_NER.csv"
        else:
          file = "2022_01_" + dia + "_" + parte + "_" + table + ".csv"

        df = pd.read_csv("https://raw.githubusercontent.com/lopezbec/COVID19_Tweets_Dataset/main/" + table + "/2022_01/" + file, dtype={'Country': str})

        if 'Country' in df:
          df = df.drop('Country', axis=1)
        df = df.dropna()
        if 'Tweet_ID' in df:
          df = df.astype({'Tweet_ID': str})
        for col in df.columns:
          df = df.rename(columns={col: col.replace(" ", "_")})

        if not os.path.exists(path + "COVID19_Tweets_Dataset_Pq3/" + table):
          os.mkdir(path + "COVID19_Tweets_Dataset_Pq3/" + table)

        df.to_parquet(
          path=path+"COVID19_Tweets_Dataset_Pq3/" + table + "/" + f'{file[:-4]}.parquet',
          engine='pyarrow',
          compression='gzip',
        )
      except:
        continue

# df = pd.read_csv("https://raw.githubusercontent.com/lopezbec/COVID19_Tweets_Dataset/main/Summary_Details/2022_01/2022_01_" + dia + "_" + parte + "_Summary_Details.csv")
# df


In [ ]:
# !cp -r COVID19_Tweets_Dataset_Pq/Summary_Sentiment_ES drive/MyDrive/Big\ Data
# !cp -r COVID19_Tweets_Dataset_Pq/Summary_NER_ES drive/MyDrive/Big\ Data

In [ ]:
path = '/content/drive/MyDrive/Big Data/'

In [ ]:
print(os.listdir(path))

['COVID19_Tweets_Dataset_Parquet', 'COVID19_Tweets_Dataset_Pq', 'COVID19_Tweets_Dataset_Pq2']


**Modelo Machine Learning v2**

---



---



In [ ]:
from pyspark import SparkContext
sc = SparkContext(master="local[2]")
sc

In [1]:
import os
path = "drive/MyDrive/Ciclos/2023-1/Big Data/ProyectoBD/Big Data/"
print(os.listdir(path))

['COVID19_Tweets_Dataset_Pq2', 'COVID19_Tweets_Dataset_Parquet', 'COVID19_Tweets_Dataset_Pq', 'COVID19_Tweets_Dataset_Pq3']


In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
       .master("local[2]") \
       .appName("test") \
       .config("spark.driver.memory", "6g")\
       .config("spark.executor.memory", "6g")\
       .getOrCreate()

summary_details_df = spark.read.parquet(path + "COVID19_Tweets_Dataset_Pq3/Summary_Details")
summary_hashtag_df = spark.read.parquet(path + "COVID19_Tweets_Dataset_Pq3/Summary_Hashtag")
summary_mentions_df = spark.read.parquet(path + "COVID19_Tweets_Dataset_Pq3/Summary_Mentions")
summary_sentiment_df = spark.read.parquet(path + "COVID19_Tweets_Dataset_Pq3/Summary_Sentiment")
summary_ner_df = spark.read.parquet(path + "COVID19_Tweets_Dataset_Pq3/Summary_NER")
summary_sentiment_es_df = spark.read.parquet(path + "COVID19_Tweets_Dataset_Pq3/Summary_Sentiment_ES")
summary_ner_es_df = spark.read.parquet(path + "COVID19_Tweets_Dataset_Pq3/Summary_NER_ES")

In [3]:
from pyspark.sql.functions import col

joined_df = summary_details_df.select("Tweet_ID", "Language", "Likes", "Retweets") \
    .join(summary_sentiment_df.select("Tweet_ID", "Sentiment_Label"), "Tweet_ID") \
    .join(summary_ner_df.select("Tweet_ID", "NER_text", "NER_Label"), "Tweet_ID")

In [4]:
joined_df = joined_df.limit(10000)
joined_df.count()

10000

In [5]:
from pyspark.ml.feature import StringIndexer, VectorAssembler, OneHotEncoder
from pyspark.ml import Pipeline

# Convertir las etiquetas de sentimiento a valores numéricos
label_indexer = StringIndexer(inputCol="Sentiment_Label", outputCol="label")

# Convertir las columnas categóricas en índices numéricos
indexers = [StringIndexer(inputCol=col, outputCol=col+"_index").setHandleInvalid("keep") for col in ["Language", "NER_text", "NER_Label"]]

# Codificar los índices numéricos en representaciones "one-hot"
encoders = [OneHotEncoder(inputCol=col+"_index", outputCol=col+"_encoded") for col in ["Language", "NER_text", "NER_Label"]]

# Seleccionar las características relevantes y combinarlas en una sola columna
feature_cols = ["Language_encoded", "NER_text_encoded", "Likes", "Retweets", "NER_Label_encoded"]
assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")

# Crear el pipeline de transformación de datos
pipeline = Pipeline(stages=indexers + encoders + [assembler, label_indexer])

# Aplicar el pipeline para transformar los datos
transformed_data = pipeline.fit(joined_df).transform(joined_df)

# Dividir los datos en conjuntos de entrenamiento y prueba
(training_data, testing_data) = transformed_data.randomSplit([0.7, 0.3], seed=123)

In [6]:
from pyspark.ml.classification import RandomForestClassifier

# Crear una instancia del clasificador
rf = RandomForestClassifier(labelCol="label", featuresCol="features", numTrees=10)

# Entrenar el modelo con los datos de entrenamiento
model = rf.fit(training_data)

In [7]:
# Realizar predicciones en el conjunto de prueba
predictions = model.transform(testing_data)

# Mostrar algunas predicciones
predictions.select("Sentiment_Label", "prediction", "probability").show(10)

+---------------+----------+--------------------+
|Sentiment_Label|prediction|         probability|
+---------------+----------+--------------------+
|        neutral|       1.0|[0.42244810571732...|
|        neutral|       0.0|[0.45632943034949...|
|        neutral|       1.0|[0.44624937856696...|
|       negative|       1.0|[0.44624937856696...|
|       negative|       1.0|[0.44624937856696...|
|       positive|       1.0|[0.42244810571732...|
|       negative|       0.0|[0.47787013761230...|
|       negative|       1.0|[0.44624937856696...|
|        neutral|       1.0|[0.42244810571732...|
|       negative|       0.0|[0.45632943034949...|
+---------------+----------+--------------------+
only showing top 10 rows



In [8]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Evaluar el rendimiento del modelo en el conjunto de prueba
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Accuracy:", accuracy)

Accuracy: 0.5117416829745597


**Modelo Machine Learning**

---



---



In [ ]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler, StringIndexer
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline

In [ ]:
spark = SparkSession.builder \
    .appName("Mi aplicacion PySpark") \
    .getOrCreate()

In [ ]:
df_summary_details = spark.read.parquet(path + "COVID19_Tweets_Dataset_Pq2/Summary_Details")
df_summary_hashtag = spark.read.parquet(path + "COVID19_Tweets_Dataset_Pq2/Summary_Hashtag")
df_summary_mentions = spark.read.parquet(path + "COVID19_Tweets_Dataset_Pq2/Summary_Mentions")
df_summary_ner = spark.read.parquet(path + "COVID19_Tweets_Dataset_Pq2/Summary_NER/")
# df_summary_ner_es = spark.read.parquet(path + "COVID19_Tweets_Dataset_Pq2/Summary_NER_ES")
df_summary_sentiment = spark.read.parquet(path + "COVID19_Tweets_Dataset_Pq2/Summary_Sentiment")
# df_summary_sentiment_es = spark.read.parquet(path + "COVID19_Tweets_Dataset_Pq2/Summary_Sentiment_ES")

In [ ]:
print(df_summary_details)
print(df_summary_hashtag)
print(df_summary_mentions)
print(df_summary_ner)
# print(df_summary_ner_es)
print(df_summary_sentiment)
# print(df_summary_sentiment_es)

DataFrame[Tweet_ID: string, Language: string, Geolocation_coordinate: string, RT: string, Likes: bigint, Retweets: bigint, Date_Created: string]
DataFrame[Tweet_ID: string, Hastag: string]
DataFrame[Tweet_ID: string, Mentions: string]
DataFrame[Tweet_ID: string, NER_Text: string, Start_Pos: bigint, Eng_Pos: bigint, NER_Label: string, Prob: double]
DataFrame[Tweet_ID: string, Sentiment_Label: string, Logits_Neutral: double, Logits_Positive: double, Logits_Negative: double]


In [ ]:
df_summary_details.head(1)

[Row(Tweet_ID='1479105481037672457', Language='es', Geolocation_coordinate='NO', RT='YES', Likes=0, Retweets=1, Date_Created='Thu Jan 06 15:00:01 +0000 2022')]

In [ ]:
# Realizar un join entre los DataFrames
df_join = df_summary_details.join(df_summary_hashtag, on='Tweet_ID')
df_join = df_join.join(df_summary_ner, on='Tweet_ID')
df_join = df_join.join(df_summary_sentiment, on='Tweet_ID')

df_join.describe()
# Seleccionar las columnas relevantes para la consulta
# df_result = df_join.select('Tweet_ID', 'Language', 'Geolocation_coordinate', 'RT', 'Likes', 'Retweets',
                          #  'Date_Created', 'Hashtag', 'NER_Text', 'Sentiment_Label')

DataFrame[summary: string, Tweet_ID: string, Language: string, Geolocation_coordinate: string, RT: string, Likes: string, Retweets: string, Date_Created: string, Hastag: string, NER_Text: string, Start_Pos: string, Eng_Pos: string, NER_Label: string, Prob: string, Sentiment_Label: string, Logits_Neutral: string, Logits_Positive: string, Logits_Negative: string]

In [ ]:
df_join = df_join.select('Tweet_ID', 'Likes', 'Date_Created', 'Hastag', 'NER_Text', 'Sentiment_Label')

Row(Tweet_ID='1477067087365423104', Likes=847, Date_Created='Sat Jan 01 00:00:10 +0000 2022', Hastag='#Covid19', NER_Text='un', Sentiment_Label='neutral')

In [ ]:
# Preparación de datos
# selected_df = df_summary_sentiment.select('Logits_Neutral', 'Logits_Positive', 'Logits_Negative', 'Sentiment_Label')
label_indexer = StringIndexer(inputCol='Sentiment_Label', outputCol='label')
indexed_df = label_indexer.fit(df_join).transform(df_join)
# indexed_df = label_indexer.fit(selected_df).transform(selected_df)
# indexed_df.head(20)

In [ ]:
# Creación de conjunto de entrenamiento y prueba
train_data, test_data = indexed_df.randomSplit([0.7, 0.3])

In [ ]:
# Construcción del pipeline
# assembler = VectorAssembler(inputCols=['Logits_Neutral', 'Logits_Positive', 'Logits_Negative'], outputCol='features')
assembler = VectorAssembler(inputCols=['Likes'], outputCol='features')
lr = LogisticRegression(featuresCol='features', labelCol='label')
pipeline = Pipeline(stages=[assembler, lr])

In [ ]:
# Ajuste del modelo
model = pipeline.fit(train_data)

# Evaluación del modelo
predictions = model.transform(test_data)

# Evaluar las predicciones utilizando métricas de clasificación


In [ ]:
# Realización de predicciones en nuevos datos
new_data = spark.read.parquet(path + "COVID19_Tweets_Dataset_Pq2/Summary_Details/2022_01_01_01_Summary_Details.parquet")
new_predictions = model.transform(new_data)
new_predictions.head(10)

[Row(Tweet_ID='1477090842997313539', Language='en', Geolocation_coordinate='NO', RT='YES', Likes=0, Retweets=4, Date_Created='Sat Jan 01 01:34:34 +0000 2022', features=DenseVector([0.0]), rawPrediction=DenseVector([0.5337, 0.2632, -0.7969]), probability=DenseVector([0.4932, 0.3764, 0.1304]), prediction=0.0),
 Row(Tweet_ID='1477093031853510657', Language='und', Geolocation_coordinate='NO', RT='NO', Likes=0, Retweets=0, Date_Created='Sat Jan 01 01:43:16 +0000 2022', features=DenseVector([0.0]), rawPrediction=DenseVector([0.5337, 0.2632, -0.7969]), probability=DenseVector([0.4932, 0.3764, 0.1304]), prediction=0.0),
 Row(Tweet_ID='1477082164919521283', Language='fr', Geolocation_coordinate='NO', RT='YES', Likes=0, Retweets=79, Date_Created='Sat Jan 01 01:00:05 +0000 2022', features=DenseVector([0.0]), rawPrediction=DenseVector([0.5337, 0.2632, -0.7969]), probability=DenseVector([0.4932, 0.3764, 0.1304]), prediction=0.0),
 Row(Tweet_ID='1477082188248190984', Language='en', Geolocation_coord